In [3]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import scipy
from scipy.optimize import curve_fit
from itertools import cycle
from datetime import datetime
from datetime import timedelta
from scipy.interpolate import UnivariateSpline

In [4]:
%cd /Volumes/SEB_USB/Andrea_Thermal_2020/Manual/

/Volumes/SEB_USB/Andrea_Thermal_2020/Manual


In [5]:
JSON_df = pd.read_csv('problem_child.csv')

In [6]:
del JSON_df['Unnamed: 0']

In [7]:
def AZmet(date):
    year = date[2:4]

    AZmet_data = pd.read_csv(
        f"https://cals.arizona.edu/azmet/data/06{year}rh.txt",
        names=[
            "Year",
            "Day",
            "Hour",
            "Air Temperature",
            "Relative Humidity",
            "VPD",
            "Solar Radiation",
            "Precipitation",
            "4 inch Soil T",
            "12 inch Soil T",
            "Avg Wind Speed",
            "Wind Vector Magnitude",
            "Wind Vector Direction",
            "Wind Direction STDEV",
            "Max Wind Speed",
            "Reference Evapotranspiration",
            "Actual Vapor Pressure",
            "Dewpoint",
        ],
    )
    print("Document downloaded, loaded")
    AZmet_df = pd.DataFrame(AZmet_data)

    AZmet_df["combined"] = AZmet_df["Year"] * 1000 + AZmet_df["Day"]
    AZmet_df["date"] = pd.to_datetime(AZmet_df["combined"], format="%Y%j")
    AZmet_df = AZmet_df.set_index("date")
    
    del AZmet_df["combined"]
    
    return AZmet_df

In [8]:
def find_date(AZmet_df, date):

    previous_day = str((pd.to_datetime(date) - timedelta(days=1)).date())
    next_day = str((pd.to_datetime(date) + timedelta(days=1)).date())

    yesterday = AZmet_df[AZmet_df.index == previous_day]
    today = AZmet_df[AZmet_df.index == date]
    tomorrow = AZmet_df[AZmet_df.index == next_day]

    concat = pd.concat([yesterday, today]).reset_index()
    concat_all = pd.concat([concat, tomorrow.reset_index()]).reset_index()

    Hour_0_index = concat[concat["date"] == date].index[0] - 1
    Hour_25_index = concat[concat["date"] == date].index[-1] + 1

    Hour_0 = pd.DataFrame(concat.iloc[Hour_0_index]).transpose()
    Hour_25 = pd.DataFrame(concat_all.iloc[Hour_25_index]).transpose()

    date_of_interest_pre = pd.concat([Hour_0, today.reset_index()])
    date_of_interest = pd.concat([date_of_interest_pre, Hour_25])

    date_of_interest = date_of_interest.reset_index()
    del date_of_interest["index"]
    del date_of_interest["level_0"]

    date_of_interest["date"][0] = date
    date_of_interest["Hour"][0] = 0

    date_of_interest["date"][25] = date
    date_of_interest["Hour"][25] = 25

    return date_of_interest

In [9]:
def splines(
    df, xvar, yvar
):  # xdata would be the information you want to use ex: df['Hour']
    xdata = df[xvar]
    ydata = df[yvar]
    x, y = xdata.values, ydata.values
    spl = UnivariateSpline(x, y)
    # spl.set_smoothing_factor(50)

    xrange = np.arange(0, 24, 0.01667)

    d = {"Minute": np.arange(len(xrange)), yvar: spl(xrange)}
    finer_df = pd.DataFrame(data=d)

    hour_list = np.arange(0, 24, 1)

    K = 60
    res = [ele for ele in hour_list for i in range(K)]

    finer_df["Hour"] = res

    minute_cycle = cycle(np.arange(0, 60, 1))
    finer_df["Minute"] = [next(minute_cycle) for cycle in range(len(finer_df))]

    year = df["Year"].unique()
    date = df["date"].unique()
    finer_df["Year"] = year[0]
    finer_df["date"] = date[0]

    finer_df["Hour"] = pd.to_timedelta(finer_df["Hour"], unit="h")
    finer_df["Minute"] = pd.to_timedelta(finer_df["Minute"], unit="m")

    finer_df["date"] = finer_df["date"] + finer_df["Hour"] + finer_df["Minute"]
    return finer_df

In [10]:
def retrieve_splines(df):
    temp_df = splines(df, "Hour", "Air Temperature")
    temp_df["VPD"] = splines(df, "Hour", "VPD")["VPD"]
    temp_df["Relative Humidity"] = splines(df, "Hour", "Relative Humidity")[
        "Relative Humidity"
    ]
    temp_df["Avg Wind Speed"] = splines(df, "Hour", "Avg Wind Speed")["Avg Wind Speed"]
    temp_df["Solar Radiation"] = splines(df, "Hour", "Solar Radiation")[
        "Solar Radiation"
    ]
    return temp_df

In [11]:
def AZMget(JSON_df):

    season = 'season_11_sorghum_yr_2020'
    # date = args.date

    # Finds unique dates
    date_list = []
    JSON_df['time'] = pd.to_datetime(JSON_df['time'])
    dates = JSON_df["time"].dt.date.unique()
    for date in dates:
        date_list.append(date)

    if len(date_list) > 1:
        date1 = str(date_list[0])
        date2 = str(date_list[1])

        AZmet_date1 = AZmet(date1)
        AZmet_date2 = AZmet(date2)

        if AZmet_date1.equals(AZmet_date2) == False:
            AZmet_df = AZmet_df1
            AZmet_df = pd.concat([AZmet_date1, AZmet_date2])
        else:
            AZmet_df = AZmet_date1

        date_of_interest1 = find_date(AZmet_df, date1)
        date_of_interest2 = find_date(AZmet_df, date2)

        temp_df1 = retrieve_splines(date_of_interest1)
        temp_df2 = retrieve_splines(date_of_interest2)

        temp_df = pd.concat([temp_df1, temp_df2])

    else:
        date1 = str(date_list[0])
        AZmet_df = AZmet(date1)

        date_of_interest = find_date(AZmet_df, date1)

        temp_df = retrieve_splines(date_of_interest)

    temp_df["Hour"] = temp_df["date"].dt.hour
    temp_df["Minute"] = temp_df["date"].dt.minute

    date_of_interest = temp_df
    print("* * AZmet Date of Interest Gathered * *")

    # image_file = JSON_df
    JSON_df["time"] = pd.to_datetime(JSON_df["time"])
    JSON_df = pd.DataFrame(JSON_df)

    print("* * JSON_df Formatted * *")

    # EnvLog = Env_data()
    # EnvLog = EnvLog.set_index("Time")
    # EnvLog = EnvLog.reset_index()

    # print("* * Environment Logger Date of Interest Gathered * *")

    return JSON_df, date_of_interest

In [12]:
def azmet_dict(JSON_df):
    AZmet_dict = {}
    JSON_df, date_of_interest = AZMget(JSON_df)
    cnt = 0
    for i, row in JSON_df.iterrows():
        cnt += 1
        time = row["time"]
        timestamp = time.round("min")
        result_index = date_of_interest[
            date_of_interest["date"] == timestamp
        ].index.values[0]
        # result_index_env = EnvLog[EnvLog["Time"] == timestamp].index.values[0]
        # result_index = date_of_interest["date"].sub(time).abs().idxmin()
        # result_index_env = EnvLog["Time"].sub(time).abs().idxmin()
        #         time = row['time'].round('H')
        #         result_index = time.hour
        AZmet_temp = date_of_interest["Air Temperature"].iloc[result_index]
        AZmet_wind = date_of_interest["Avg Wind Speed"].iloc[result_index]
        AZmet_vpd = date_of_interest["VPD"].iloc[result_index]
        AZmet_solar = date_of_interest["Solar Radiation"].iloc[result_index]
        AZmet_rh = date_of_interest["Relative Humidity"].iloc[result_index]
        # Env_temp = EnvLog["Temperature"].iloc[result_index_env]
        # Env_wind = EnvLog["Wind velocity"].iloc[result_index_env]
        AZmet_dict[cnt] = {
            "azmet_atm_temp": AZmet_temp,
            "azmet_wind_velocity": AZmet_wind,
            "azmet_VPD": AZmet_vpd,
            "azmet_solar_radiation": AZmet_solar,
            "relative_humidity": AZmet_rh,
            # "env_temp": Env_temp,
            # "env_wind": Env_wind,
        }
    return pd.DataFrame.from_dict(AZmet_dict)

In [13]:
def compile_info(JSON_df, environmental_df):
    JSON_df["azmet_atm_temp"] = environmental_df["azmet_atm_temp"]
    JSON_df["azmet_wind_velocity"] = environmental_df["azmet_wind_velocity"]
    JSON_df["azmet_VPD"] = environmental_df["azmet_VPD"]
    JSON_df["azmet_solar_radiation"] = environmental_df["azmet_solar_radiation"]
    JSON_df["relative_humidity"] = environmental_df["relative_humidity"]
    # JSON_df["env_temp"] = environmental_df["env_temp"]
    # JSON_df["env_wind"] = environmental_df["env_wind"]
    return JSON_df

In [14]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace("[", "").replace("]", "")

In [15]:
def expand_plots(clean_file):
    file = clean_file
    file["plot"] = (file["plot"].apply(listToStringWithoutBrackets)).apply(eval)
    plot_expand = file["plot"].apply(pd.Series)
    plot_expand["time"] = file["time"]
    plot_expand["Image Name"] = file["filename"]
    # plot_expand["env_temp"] = file["env_temp"]
    # plot_expand["env_wind"] = file["env_wind"]
    plot_expand["azmet_atm_temp"] = file["azmet_atm_temp"]
    plot_expand["azmet_wind_velocity"] = file["azmet_wind_velocity"]
    plot_expand["azmet_VPD"] = file["azmet_VPD"]
    plot_expand["azmet_solar_radiation"] = file["azmet_solar_radiation"]
    plot_expand["relative_humidity"] = file["relative_humidity"]
    stacked = plot_expand.set_index(
        [
            "time",
            "Image Name",
            # "env_temp",
            # "env_wind",
            "azmet_atm_temp",
            "azmet_wind_velocity",
            "azmet_VPD",
            "azmet_solar_radiation",
            "relative_humidity",
        ]
    ).stack()
    stack_df = pd.DataFrame(stacked).reset_index()
    del stack_df["level_7"]
    final_df = stack_df.rename(columns={0: "Plot"})
    return final_df

In [16]:
environmental_df = azmet_dict(JSON_df).transpose()

Document downloaded, loaded
* * AZmet Date of Interest Gathered * *
* * JSON_df Formatted * *


/var/folders/gp/54shw8hs061c_77pj8zjvvfc0000gn/T/ipykernel_42190/1051362446.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_of_interest["date"][0] = date
/var/folders/gp/54shw8hs061c_77pj8zjvvfc0000gn/T/ipykernel_42190/1051362446.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_of_interest["Hour"][0] = 0
/var/folders/gp/54shw8hs061c_77pj8zjvvfc0000gn/T/ipykernel_42190/1051362446.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [17]:
file = compile_info(JSON_df, environmental_df)

In [18]:
finale_df = expand_plots(file)

In [19]:
env_logger = JSON_df[
    [
        "time",
        "filename",
        "azmet_atm_temp",
        "azmet_wind_velocity",
        "azmet_VPD",
        "azmet_solar_radiation",
        "relative_humidity",
        # "env_temp",
        # "env_wind",
    ]
].set_index("filename")

In [20]:
img_plot = finale_df[["Image Name", "Plot"]]
img_plot.columns = ["image", "plot"]

img_plot["Date and Time"] = img_plot["azmet_atm_temp"] = img_plot[
    "azmet_wind_velocity"
] = img_plot["azmet_VPD"] = img_plot["azmet_solar_radiation"] = img_plot[
    "relative_humidity"
] = None

/var/folders/gp/54shw8hs061c_77pj8zjvvfc0000gn/T/ipykernel_42190/465291622.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img_plot["Date and Time"] = img_plot["azmet_atm_temp"] = img_plot[
/var/folders/gp/54shw8hs061c_77pj8zjvvfc0000gn/T/ipykernel_42190/465291622.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ] = img_plot["azmet_VPD"] = img_plot["azmet_solar_radiation"] = img_plot[


In [21]:
for i, row in img_plot.iterrows():
    meta = row["image"]
    try:
        datetime = env_logger.loc[meta, "time"]
        azmet_temp = env_logger.loc[meta, "azmet_atm_temp"]
        azmet_wind_vel = env_logger.loc[meta, "azmet_wind_velocity"]
        azmet_vpd = env_logger.loc[meta, "azmet_VPD"]
        sol_rad = env_logger.loc[meta, "azmet_solar_radiation"]
        # temp = env_logger.loc[meta, "env_temp"]
        # win_vel = env_logger.loc[meta, "env_wind"]
        rel_hum = env_logger.loc[meta, "relative_humidity"]

        img_plot.at[i, "Date and Time"] = datetime
        img_plot.at[i, "azmet_atm_temp"] = azmet_temp
        img_plot.at[i, "azmet_wind_velocity"] = azmet_wind_vel
        img_plot.at[i, "azmet_VPD"] = azmet_vpd
        img_plot.at[i, "azmet_solar_radiation"] = sol_rad
        # img_plot.at[i, "env_temp"] = temp
        # img_plot.at[i, "env_wind"] = win_vel
        img_plot.at[i, "relative_humidity"] = rel_hum

    except:
        pass

In [22]:
df_agg = img_plot.drop(["image", "Date and Time"], axis=1)

## Manually adding the data to the csv

In [479]:
result_path = 'indiv_temps/Result_csv/'
plant_detection_path = 'indiv_temps/2020-09-15__12-25-49-034-individual_thermal.csv'

result = pd.read_csv(result_path)
plant_detections = pd.read_csv(plant_detection_path)

In [480]:
plant_detections['plot'] = plant_detections['plot'].astype(str).str.zfill(4)
result['plot'] = result['plot'].astype(str).str.zfill(4)

In [481]:
plant_detections = plant_detections.groupby('plot').mean().reset_index()

In [482]:
del result['Unnamed: 0']
result = result.reset_index()
del result['index']

In [483]:
result = result.sort_values(by = 'plot').set_index('plot')

In [484]:
plant_detections = plant_detections.sort_values(by = 'plot')

In [485]:
# plot = plant_detections.iloc[2]['plot']
# plant_temp = plant_detections.iloc[2]['median']

# atm_temp = result.loc[plot, "median"]
# norm_temp = atm_temp - plant_temp

# azmet_wind_vel = result.loc[plot, "azmet_wind_velocity"]

# plant_detections.at[i, "azmet_wind_velocity"] = azmet_wind_vel

In [486]:
plant_detections["norm_temp"] = plant_detections["atm_temp"] = plant_detections["azmet_wind_velocity"] = plant_detections["azmet_VPD"] = plant_detections["azmet_solar_radiation"] = plant_detections["relative_humidity"] = None

for i, row in plant_detections.iterrows():
    plot = row['plot']
    plant_temp = row['median']
    try:
        atm_temp = result.loc[plot, "median"]
        norm_temp = atm_temp - plant_temp

        azmet_wind_vel = result.loc[plot, "azmet_wind_velocity"]
        azmet_vpd = result.loc[plot, "azmet_VPD"]
        sol_rad = result.loc[plot, "azmet_solar_radiation"]
        rel_hum = result.loc[plot, "relative_humidity"]

        plant_detections.at[i, "norm_temp"] = norm_temp
        plant_detections.at[i, "atm_temp"] = atm_temp

        plant_detections.at[i, "azmet_wind_velocity"] = azmet_wind_vel
        plant_detections.at[i, "azmet_VPD"] = azmet_vpd
        plant_detections.at[i, "azmet_solar_radiation"] = sol_rad
        plant_detections.at[i, "relative_humidity"] = rel_hum
    except:
        pass

In [487]:
plant_detections

,plot,lon,lat,min_x,max_x,min_y,max_y,nw_lat,nw_lon,se_lat,...,median,quartile_3,variance,std_dev,norm_temp,atm_temp,azmet_wind_velocity,azmet_VPD,azmet_solar_radiation,relative_humidity
0,0201,-111.975050,33.074580,27.500000,122.357143,279.642857,390.714286,33.074577,-111.975053,33.074583,...,39.219302,42.836868,22.448558,4.481556,-1.256706,37.962596,1.464456,4.642875,1.764946,21.228903
1,0202,-111.975041,33.074578,22.687500,124.500000,323.125000,459.125000,33.074574,-111.975044,33.074581,...,37.907482,41.210166,14.945268,3.715064,0.055115,37.962596,1.464456,4.642875,1.764946,21.228903
2,0203,-111.975032,33.074575,31.555556,134.444444,367.888889,514.777778,33.074572,-111.975035,33.074579,...,37.427420,39.686977,9.171468,2.920550,0.535176,37.962596,1.464456,4.642875,1.764946,21.228903
3,0204,-111.975026,33.074575,1.555556,118.666667,383.444444,519.111111,33.074572,-111.975029,33.074578,...,36.030199,37.911477,4.996645,2.150207,1.940707,37.970906,1.464443,4.64532,1.764849,21.212859
4,0205,-111.975017,33.074578,14.571429,136.428571,320.357143,467.571429,33.074574,-111.975020,33.074581,...,37.296064,40.091056,10.804114,3.231067,0.674842,37.970906,1.464443,4.64532,1.764849,21.212859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1623,5326,-111.974847,33.076406,16.500000,132.666667,404.166667,488.500000,33.076404,-111.974851,33.076408,...,44.898254,47.259544,7.664592,2.717165,-8.622335,36.275919,1.459518,4.229502,1.75215,24.295716
1624,5327,-111.974840,33.076407,43.777778,111.333333,374.333333,448.111111,33.076406,-111.974842,33.076409,...,45.914158,49.631861,16.120559,3.881024,-9.638239,36.275919,1.459518,4.229502,1.75215,24.295716
1625,5328,-111.974832,33.076405,44.000000,74.000000,436.000000,485.000000,33.076404,-111.974833,33.076406,...,49.202654,53.397614,23.044556,4.754924,-12.926735,36.275919,1.459518,4.229502,1.75215,24.295716
1626,5331,-111.974804,33.076411,96.500000,148.500000,269.500000,380.500000,33.076409,-111.974806,33.076414,...,45.429038,50.227668,19.485774,4.375830,-9.153119,36.275919,1.459518,4.229502,1.75215,24.295716


In [488]:
name = plant_detection_path.split('/')[1]

In [489]:
plant_detections.to_csv(f'{name}_plant_detections.csv')